<a href="https://colab.research.google.com/github/voke-brume/AI-ML/blob/main/AI/ComputerVision/ImageQuantization/NaiveVsFloyd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ASSIGNMENT 1**
# Instructor: Gianfranco Doretto
# Due: September 15, 2022 by the beginning of the lecture
# Student Name: Voke Brume

The assignment is composed of two sections. The first one is made of simple programmng questions that require you to use Python. The second section requires you to implement and test two image processong algorithms.

If you are new to Colab, Python Numpy, and so on, you are strongly encouraged to read [this tutorial](https://cs231n.github.io/python-numpy-tutorial/). 

**Submission Instructions**

You will need to prepare a folder called

```hw1_YOUR-LAST-NAME_YOUR-FIRST-NAME```

In that folder you want to put
- Your Colab notebook (fill out the Student Name above)
- For every question that requires submitting data you create a subfolder called ```question_NUMBER``` and you put there the files with adequate naming. If we do not understand that something is there you will not get credit!

You will need to zip the folder and you will need to submit the zip file ```hw1_YOUR-LAST-NAME_YOUR-FIRST-NAME.zip``` to the instructor by sharing the file via Google Drive.

In order to [share a file](https://support.google.com/drive/answer/2494822?hl=en&co=GENIE.Platform%3DDesktop) with the instructor please use this email gidoretto@mix.wvu.edu

During the evaluation we are going to test your Python notebook with Google Colab. So, the assumption is that you are going to do all the development with Colab. We will run your notebook by running the cells sequentially. They will have to produce the expected result in order for you to get credit.

# *Exercise 1*
This is a Python Numpy wormup exercise. Complete each of the functions below as descibed by the Input/Returns requirements. Below each function add a code cell, and write in it just a few lines of code to show that your function works.
Every function should be implementable with only **one** line of code.

For example, this function:
```python
def f0(X):
    """
    Input:
    - X: A numpy array

    Returns:
    - A matrix Y such that Y[i, j] = X[i, j] * 10 + 100

    Hint: Trust that numpy will do the right thing
    """
    return None
```
Should be completed as
```python
def f0(X):
    """
    Input:
    - X: A numpy array

    Returns:
    - A matrix Y such that Y[i, j] = X[i, j] * 10 + 100

    Hint: Trust that numpy will do the right thing
    """
    return X*10+100
```
An example of code that tests ```f0``` could be
```python
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
print(x)
print(f0(x))
```

In [ ]:
import numpy as np

In [ ]:
def f1(X, Y):
    """
    Inputs:
    - X: A numpy array of shape (N, N)
    - Y: A numpy array of shape (N, N)

    Returns:
    A numpy array Z such that Z[i, j] = X[i, j] + 10 * Y[i, j]

    Hint: Trust that numpy will do the right thing
    """
    return np.add( X, np.multiply(10, Y) )

Add a code cell below with just enough lines that show that your function works.

In [ ]:
# Test for f1
x = np.array([ [1, 2], [3, 4] ])
y = np.array([ [5, 6], [7, 8] ])

print( f1(x, y) )

[[51 62]
 [73 84]]


In [ ]:
def f2(X, Y):
    """
    Inputs:
    - X: A numpy array of shape (N, N)
    - Y: A numpy array of shape (N, N)

    Returns:
    A numpy array Z such that Z[i, j] = X[i, j] * Y[i, j] - 10

    Hint: By analogy to +, * will do the same thing
    """
    return np.subtract( np.multiply(X, Y), 10 )

Add a code cell below with just enough lines that show that your function works.

In [ ]:
# Test for f2
x = np.array([ [1, 2], [3, 4] ])
y = np.array([ [5, 6], [7, 8] ])

print( f2(x, y) )

[[-5  2]
 [11 22]]


In [ ]:
def f3(X):
    """
    Inputs:
    - X: A numpy array of shape (N, N) of floating point numbers

    Returns:
    A numpy array with the same data as X, but cast to 32-bit integers

    Hint: astype
    """
    return X.astype('int32')

Add a code cell below with just enough lines that show that your function works.

In [ ]:
# Test for f3
x = np.array([ [1.0, 2.0], [3.0, 4.0] ])
print(x.dtype, f3(x), f3(x).dtype)

float64 [[1 2]
 [3 4]] int32


In [ ]:
def f4(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    - A numpy array Y of shape (N * M, 1) containing the entries of X in row
      order. That is, X[i, j] = Y[i * M + j, 0]

    Hint:
    1) np.reshape
    2) You can specify an unknown dimension as -1
    """
    return np.reshape(X, (-1, 1))


Add a code cell below with just enough lines that show that your function works.

In [ ]:
# Test for f4
x = np.array([ [1, 2, 3], [4, 5, 6] ])
print(f4(x))

[[1]
 [2]
 [3]
 [4]
 [5]
 [6]]


In [ ]:
def f5(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M) where each entry is between 0 and 1

    Returns:
    A numpy array Y where Y[i, j] = True if X[i, j] > 0.5

    Hint: Trust python to do the right thing
    """
    return (x > 0.5)

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for f5
x = np.array([ [0.1, 0.4, 0.6], [0.5, 0.7, 0.8] ])
print(f5(x))

[[False False  True]
 [False  True  True]]


In [ ]:
def f6(A, v):
    """
    Inputs:
    - A: A numpy array of shape (N, F)
    - v: A numpy array of shape (F, 1)

    Returns:
    Numpy array of shape (N, 1) giving the matrix-vector product Av
    """
    return A @ v

Add a code cell below with just enough lines that show that your function works.

In [ ]:
# Test for f6
a = np.array([ [1, 2, 3], [4, 5, 6] ])
v = np.array([ [7], [8], [9] ])
print(f6(a, v))

[[ 50]
 [122]]


In [ ]:
def f7(v):
    """
    Inputs:
    - v: A numpy array of shape (N, 1)

    Returns:
    The L2 norm of v: norm = (sum_i^N v[i]^2)^(1/2)
    You MAY NOT use np.linalg.norm
    """
    return np.sqrt( np.sum( np.square(v) ) )

Add a code cell below with just enough lines that show that your function works.



In [ ]:
# Test for f7
v = np.array([ [1], [2], [3], [4], [5] ])
print(f7(v))

7.416198487095663


In [ ]:
def f8(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    The sum of all entries in X

    Hint: np.sum
    """
    return np.sum(X)

Add a code cell below with just enough lines that show that your function works.



In [ ]:
# Test for f8
x = np.array([ [1, 2, 3], [4, 5, 6] ])
print(f8(x))

21


In [ ]:
def f9(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    A numpy array S of shape (M,) where S[j] is the sum of column j of X

    Hint: np.sum has an optional "axis" argument
    """
    return np.sum(X, axis=0)

Add a code cell below with just enough lines that show that your function works.



In [ ]:
# Test for f9
x = np.array([ [1, 2, 3], [4, 5, 6] ])
print(f9(x))

[5 7 9]


In [ ]:
def f10(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    A numpy array S of shape (N, 1) where S[i] is the L2 norm of row i of X
    """
    return np.reshape(np.sqrt( np.sum(np.square(X), axis=1) ), (-1, 1))

Add a code cell below with just enough lines that show that your function works.




In [ ]:
# Test for f910
x = np.array([ [1, 2, 3, 4, 5], [6, 7, 8, 9, 10] ])
print(f10(x))

[[ 7.41619849]
 [18.16590212]]


# *Exercise 2*
This is a Python Numpy exercise. Complete each of the functions below as descibed by the Input/Returns requirements. Below each function add a code cell, and write in it just a few lines of code to show that your function works.
These functions may require you to write more than one line of code. The instructions will indicate what is the expected number of lines. You might be able to write fewer lines, but you should not exceed the maximum number of lines indicated. __You are not allowed to use the ```for``` loop construct__.


In [ ]:
import numpy as np
import copy

In [ ]:
def g1(L):
    """
    Inputs:
    - L: A list of M numpy arrays, each of shape (1, N)

    Returns:
    A numpy array of shape (M, N) giving all inputs stacked together

    Lines of code: 1

    Hint: vstack/hstack/dstack, no for loop
    """
    return np.vstack(L)

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g1
l = []
l.append( np.array([1, 2, 3]) )
l.append( np.array([4, 5, 6]) )
print( l, type(l) )
print( g1(l), type( g1(l) ) )

[array([1, 2, 3]), array([4, 5, 6])] <class 'list'>
[[1 2 3]
 [4 5 6]] <class 'numpy.ndarray'>


In [ ]:
def g2(X):
    """
    Inputs:
    - X: A numpy array of any shape

    Returns:
    A copy of X, but with all negative entires set to 0

    Lines of code: 2-3

    Hint:
    1) If S is a boolean array with the same shape as X, then X[S] gives an
       array containing all elements of X corresponding to true values of S
    2) X[S] = v assigns the value v to all entires of X corresponding to
       true values of S.
    """
    copy_of_X = copy.deepcopy(X)
    return np.where(copy_of_X < 0, 0, copy_of_X )

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g2
x = np.array([ [1, -2, 3], [-4, 5, -6] ])
print(g2(x))

[[1 0 3]
 [0 5 0]]


In [ ]:
def g3(X):
    """
    Inputs:
    - X: A numpy array of shape (N, N)

    Returns:
    A numpy array of shape (4, 4) giving the upper left 4x4 submatrix of X
    minus the bottom right 4x4 submatrix of X.

    Lines of code: 1-2

    Hint:
    1) X[y0:y1, x0:x1] gives the submatrix
       from rows y0 to (but not including!) y1
       from columns x0 (but not including!) x1
    """
    return np.subtract( X[:4, :4], X[-4:, -4:] )

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g3
x = np.reshape(np.arange(1, 129), (8, -1))
print(x)
print(g3(x))

[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16]
 [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32]
 [ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48]
 [ 49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64]
 [ 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80]
 [ 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96]
 [ 97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112]
 [113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128]]
[[-76 -76 -76 -76]
 [-76 -76 -76 -76]
 [-76 -76 -76 -76]
 [-76 -76 -76 -76]]


In [ ]:
def g4(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    A numpy array Y of the same shape as X, where Y[i] is a vector that points
    the same direction as X[i] but has unit norm.

    Lines of code: 1-3

    Hints:
    1) The vector v / ||v||| is the unit vector pointing in the same direction
       as v (as long as v != 0)
    2) Divide X by a vector of normalization factors, of shape (N, 1)
    3) Elementwise operations between an array of shape (N, M) and an array of
       shape (N, 1) work -- try it! This is called "broadcasting"
    4) Elementwise operations between an array of shape (N, M) and an array of
       shape (N,) won't work -- try reshaping
    """
    return X / np.reshape( np.sqrt(np.sum(np.square(X), axis=1)), (len(np.sqrt(np.sum(np.square(X), axis=1))), 1) ) 

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g4
x = np.array([ [1, 2, 3, 4, 5], [6, 7, 8, 9, 10] ])
print(g4(x), g4(x).shape)

[[0.13483997 0.26967994 0.40451992 0.53935989 0.67419986]
 [0.33028913 0.38533732 0.44038551 0.49543369 0.55048188]] (2, 5)


In [ ]:
def g5(q, k, v):
    """
    Inputs:
    - q: A numpy array of shape (1, K) (queries)
    - k: A numpy array of shape (N, K) (keys)
    - v: A numpy array of shape (N, 1) (values)

    Returns:
    sum_i exp(-||q-k_i||^2) * v[i]

    Lines of code: 1-3

    Hints:
    1) You can perform elementwise operations on arrays of shape (N, K) and
       (1, K) with broadcasting
    2) Recall that np.sum has useful "axis" and "keepdims" options
    3) np.exp and friends apply elementwise to arrays
    """
    return np.sum(np.exp(-1*np.sqrt(np.sum((q-k)**2, axis=1))**2)*v, axis=1)

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g5
q = np.array([5, 2, 7])
k = np.array([ [3, 17, 4], [13, 6, 24] ])
v = np.array([ [23], [37] ])
print(g5(q, k, v))

[9.99174043e-103 1.60736694e-102]


In [ ]:
def g6(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    A numpy array D of shape (N, N) where D[i, j] gives the Euclidean distance
    between X[i] and X[j], using the identity
    ||x - y||^2 = ||x||^2 + ||y||^2 - 2x^T y

    Line of code: 2-3

    Hints:
    1) What happens when you add two arrays of shape (1, N) and (N, 1)?
    2) Think about the definition of matrix multiplication
    3) Transpose is your friend
    4) Note the square! Use a square root at the end
    5) On some machines, ||x||^2 + ||x||^2 - 2x^Tx may be slightly negative,
       causing the square root to crash. Just take max(0, value) before the
       square root. Seems to occur on Macs.
    """

    x_diff = X[:, np.newaxis, :] - X[np.newaxis, :, :]
    return (x_diff * x_diff).sum(axis=2)

In [ ]:
# Test for g6
x = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
print(g6(x))

[[  0  27 108 243]
 [ 27   0  27 108]
 [108  27   0  27]
 [243 108  27   0]]


Add a code cell below with just enough lines that show that your function works.


In [ ]:
def g7(q, V):
    """
    Inputs:
    - q: A numpy array of shape (1, M) (query)
    - V: A numpy array of shape (N, M) (values)

    Return:
    The index i that maximizes the dot product q . V[i]

    Lines of code: 1

    Hint: np.argmax
    """
    return np.argmax(V.dot(q))


Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g7
v = np.array([[1, 2, 3], [3, 2, 1]])
q = np.array([6, 5, 4])
print(g7(q, v))

1


In [ ]:
def g8(X, Y):
    """
    Inputs:
    - X: A numpy array of shape (N, 3)
    - Y: A numpy array of shape (N, 3)

    Returns:
    A numpy array C of shape (N, 3) such C[i] is the cross product between X[i]
    and Y[i]

    Lines of code: 1

    Hint: np.cross
    """
    return np.cross(X, Y)

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g8
x = np.array([ [1, 2, 3], [4, 5, 6] ])
y = np.array([ [7, 8, 9], [10, 11, 12] ])
print(g8(x, y))

[[-6 12 -6]
 [-6 12 -6]]


In [ ]:
def g9(X):
    """
    Inputs:
    - X: A numpy array of shape (N, M)

    Returns:
    A numpy array Y of shape (N, M + 1) such that
        Y[i, :F] = X[i]
        Y[i, F] = 1

    Lines of code: 1

    Hint: np.hstack, np.ones
    """
    return np.hstack( ( X, np.ones((X.shape[0], 1)) ) )

Add a code cell below with just enough lines that show that your function works.


In [ ]:
# Test for g9
x = np.array([ [1, 2, 3], [4, 5, 6], [7, 8, 9] ])
print(g9(x))

[[1. 2. 3. 1.]
 [4. 5. 6. 1.]
 [7. 8. 9. 1.]]


In [ ]:
def g10(N, s, x, y):
    """
    Inputs:
    - N: An integer
    - s: A floating-point number
    - x: A floating-point number
    - y: A floating-point number

    Returns:
    A numpy array I of shape (N, N) such that
    I[i, j] = exp(-||(j, i) - (x, y)||^2 / s^2)

    Lines of code: 2-3

    Hint: Be careful with types -- float and int aren't the same!
    """
    return None

Add a code cell below with just enough lines that show that your function works.


# *Exercise 3*

In this exercise you are going to experiment with image quantization. 

You have been given two folders with images. The folder ```images``` contains a gallery of 10 images. The folder ```images_200``` contains the same images but resized with the biggest dimension to not exceed 200 pixels. Initially, you will operate on the folder with the smaller images because the algorithms you will implement will be relatively slow, and will take a lot of time if made them to work on the images with original size. Also, you will convert the images to grayscale.

Later in the exercise, you will implement a resizing function that will allow you to operate at the desired resolution, and you will also extend the algorithms to handle colors.

The idea of image quantization is that the 256 levels used to represent each color channel or the intensity (if the image is grayscale), have to be mapped according to a uniform palette, obtained by setting the number of bits per color channel that you use. To encode the original 256 levels, 8 bits are necessary. To encode in 2 levels only 1 bit is necessary.

In this exercise you will be concerned to mainly generate quantization results based on 1 bit and on 2 bits.

In principle, encoding an image with less bits per pixel allows to represent it more efficiently, and it enables its display in mediums that have a limited palette. One example is the [GIF](https://en.wikipedia.org/wiki/GIF) image format, which allows a palette of only 256 colors, as opposed to the full range of colors of a 24-bits image.  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
import cv2
import numpy as np

def coverPalette(N):
    # Return the palette we're using
    return np.linspace(1, 0, 2 ** N)

def reconstructImage(IQ, palette):
    """
    Given a quantized image IQ and their value, return a floating point image
    """
    # opencv is full of these assertions.
    # If you're looking here you may have an error.
    # Check to see what the assertion says
    assert(np.issubdtype(IQ.dtype, np.integer))
    return palette[IQ]

def upscaleNN(I, target_size):
    """
    NN upsample I until it hits a target size but without going over 4096
    """
    h, w = I.shape[:2]
    scale = 1
    while True:
        if min(h * scale, w * scale) >= target_size:
            break
        if max(h * (scale + 1), w * (scale + 1)) > 4096:
            break
        scale += 1
    # usually you should call a library but providing the call here defeats
    # the purpose :)
    shape = (scale, scale) if I.ndim == 2 else (scale, scale, 1)
    return np.kron(I, np.ones(shape))


The code cell below is where you are supposed to write almost all the code. You need to complete the two quantization algorithms ```quantizeNaive``` and ```quantizeFloyd```, and a utility function ```quantize``` to be used by the algorithms.

Every quantization algorithm has the following Input/Output specification:
- **Input:** (a) a $H\timesW$ floating point image with brightness ranging from 0 to 1, and (b) a palette consisting of all the $K$ allowed brightness settings (which are floats in the range 0 to 1).
- **Output:** each algorithm produces a $H\times W$ ```uint8``` image with brightness ranging from $0$ to
$K-1$. This is the quantized image. We can use the quantized image and the palette to compute a reconstructed image, and this is done by the  function ```reconstructImage```.



**Naive Quantization**

Begin by implementing ```quantize(v,palette)```. This should return the index of the nearest value in the palette to the single value ```v```. Do not use a ```for``` loop. You can look at functions like ```np.argmin```. If you find another way that is more clever, feel free to propose it.

Implement ```quantizeNaive(IF,palette)```. This approach aims at approximating the image by quantizing each pixel with the closest value in the palette. Therefore, ```quantizeNaive(IF,palette)``` takes a floating point image of size $H\times W$ and a palette of values, and creates a new ```uint8``` image  and uses ```quantize()``` to find the index of the nearest pixel. It returns the $H\times W$ ```uint8``` image of the palette indices.

**Question 1.** When you run the quantization on the full set of images in ```images_200```, in one sentence say why might your code take a very long time.

**Question 2.** For 1 and then for 2 bits per pixel, embed (show) in the cell below three results of inputs and outputs. Also, make sure you save the same images in the appropriate folder because you will need to submit them.

**How to embed images in Colab:** From your Google Drive navigate to the file, and by right-clicking on it select Get Link. Now change the option from "Restricted" to "Anyone with the link" and copy the link. You will get a link like this

```https://drive.google.com/file/d/1Ljc932JoNfmChw_CvDBbogfyht8wakIZ/view?usp=sharing```

You want to get the ID of the file:

```1YZPSVXP8s6FpkpuCSJdBymhRhfDiac58```

and put it in front of 

```https://drive.google.com/uc?id=```

to obtain the final link

```https://drive.google.com/uc?id=1Ljc932JoNfmChw_CvDBbogfyht8wakIZ```

Now you can embed the image by using ```![](link)```
This is how it is done here:

![](https://drive.google.com/uc?id=1Ljc932JoNfmChw_CvDBbogfyht8wakIZ)



**Answer to Question 1:**
The code may take a very long time because the naive quantizazer implemented is a brute force method that requires too much computation by computing the smallest representative in the palette for each pixel in the original image, thereby making it a linear process.



**Answer to Question 2:**

Grayscale Quantize Naive 1-Bit Images

![](https://drive.google.com/uc?id=1-RDcK4k14gxtnTEJVpHeNu4SGOM6JCb8) 
![](https://drive.google.com/uc?id=1-GGJs4GcunL6EIxl1UnEUgffz5aeZv7X)
![](https://drive.google.com/uc?id=1-ZVZIpBxrZtVTaDVqO_Dl-4WDI6wUNhH) 
![](https://drive.google.com/uc?id=1-YgElFji8ULxRbAGJOyh8P1ujtSBAFJ2)
![](https://drive.google.com/uc?id=100xRQcyvkQkzunDi3Uki3U2eXL025L6x) 
![](https://drive.google.com/uc?id=1-vjHArF0v80lEIpel5aNCcZLdY1nf9NZ)

[View all results for quantizeNaive 1-bit](https://drive.google.com/uc?id=10MCyd2tio3eYBx6c6xjCNQD09Dw5nLvP)


Grayscale Quantize Naive 2-Bit Images

![](https://drive.google.com/uc?id=1-RDcK4k14gxtnTEJVpHeNu4SGOM6JCb8) 
![](https://drive.google.com/uc?id=10Pp2y6uLxSjBKQ4kbaOXXNaARBpSDfVF)
![](https://drive.google.com/uc?id=1-ZVZIpBxrZtVTaDVqO_Dl-4WDI6wUNhH) 
![](https://drive.google.com/uc?id=10a4lYUYHQmiPmykT7pyfMYPCpt-ELgaG)
![](https://drive.google.com/uc?id=100xRQcyvkQkzunDi3Uki3U2eXL025L6x) 
![](https://drive.google.com/uc?id=10weB1xGWlnl1YrlxS-UJgc69V8i2Q3e8)

[View all results for quantizeNaive 2-bit](https://drive.google.com/uc?id=11WwCgCH9xsdOfuzzJUXtV5txuYKTnOx_)


**Floyd Quantization**

To improve upon the naive quantization, you will implement the [Floyd–Steinberg dithering](https://en.wikipedia.org/wiki/Floyd–Steinberg_dithering) algorithm in the function ```quantizeFloyd```.
The pseudocode from Wikipedia is summarized below:
```python
output = new HxW that indexes into the palette
for y in range(H): for x in range(W):
  oldValue = pixel[x][y]
  colorIndex = quantize(oldValue,palette)
  output[x][y] = colorIndex
  newValue = palette[colorIndex]
  error = oldValue-newValue
  pixel[x+1][y] += error*7/16
  pixel[x-1][y+1] += error*3/16
  pixel[x][y+1] += error*5/16
  pixel[x+1][y+1] += error*1/16
return Output
```
In your implementation you want to pay attention to a few important things. For instance, depending on the libraries you are using, the way in which a pixel is indexed changes, because they will make different assumptions about whether x or y come first. As you implement the algorithm there will be edge cases, and you might need to define what behavior to follow. Finally, be careful about how data is passed to the function. In order to produce the output, you will need to change the values of ```IF```. You may need to make an internal copy of it (```IF.copy()```), if necessary, since you do not want to change the data you were given as input.

**Question 3.** Explain briefly why the Floyd quantization does a better job than the naive version.

**Question 4.** For 1 and then for 2 bits per pixel, embed (show) in the cell below three results of inputs and outputs. Also, make sure you save the same images because you will need to submit them.


**Answer to Question 3:**
Floyd quantization does a better job than the naive version because it uses a more elaborate diffusion matrix to distribute the quantization error to all directly neighboring unvisited pixels. Furthermore, the numbers used prevents a lot of patterns from forming, unlike the naive quantization technique.

**Answer to Question 4:**

Grayscale Quantize Floyd 1-Bit Images

![](https://drive.google.com/uc?id=1-RDcK4k14gxtnTEJVpHeNu4SGOM6JCb8) 
![](https://drive.google.com/uc?id=124GAlWUsLj8Bu9ynpzrfoMsEhE3P4BP4)
![](https://drive.google.com/uc?id=1-ZVZIpBxrZtVTaDVqO_Dl-4WDI6wUNhH) 
![](https://drive.google.com/uc?id=12C6PB4uIxvejzLBVGZuAnyxopWQ26wCx)
![](https://drive.google.com/uc?id=100xRQcyvkQkzunDi3Uki3U2eXL025L6x) 
![](https://drive.google.com/uc?id=12J9kx29S-b0DzhBRP74lif5L-51ZlDeD)

[View all results for quantizeNaive 1-bit](https://drive.google.com/uc?id=12jY2sRhxxqgwvkA6c6RPsQ_j9NDwpKVz)


Grayscale Quantize Floyd 2-Bit Images

![](https://drive.google.com/uc?id=1-RDcK4k14gxtnTEJVpHeNu4SGOM6JCb8) 
![](https://drive.google.com/uc?id=13b5pMqkxozAfxGED5SgB213XgbtujFci)
![](https://drive.google.com/uc?id=1-ZVZIpBxrZtVTaDVqO_Dl-4WDI6wUNhH) 
![](https://drive.google.com/uc?id=13i5LltlzMYfwguHK0jeKSbwcqvzwAh_y)
![](https://drive.google.com/uc?id=100xRQcyvkQkzunDi3Uki3U2eXL025L6x) 
![](https://drive.google.com/uc?id=12mGL6uxEL2ISLiO4mKpGIg_XcpVEXHom)

[View all results for quantizeNaive 2-bit](https://drive.google.com/uc?id=13wKK5UahaQFOCQk7UhC_OHLToov7R7lB)

In [ ]:
def quantize(v, palette):
    """
    Given a scalar v and array of values palette,
    return the index of the closest value
    """
    if np.isscalar(v): return np.abs(palette - v).argmin()
    output = np.zeros( (v.shape[0]), dtype = int )
    for i in range (len(v)): output[i] = np.abs(palette - v[i]).argmin()
    return output

def quantizeNaive(IF, palette):
    """Given a floating-point image return quantized version (Naive)"""
    newImage = np.zeros(IF.shape, dtype = 'uint8')
    for i in range (IF.shape[0]):
      for j in range (IF.shape[1]):
        newImage[i][j] = quantize(IF[i][j], palette)
    return newImage

def quantizeFloyd(IF, palette):
    """
    Given a floating-point image return quantized version (Floyd-Steinberg)
    """
    output = np.zeros(IF.shape, dtype = 'uint8')
    h = IF.shape[0]
    w = IF.shape[1]
    for x in range (h):
      for y in range (w):
        old_value = IF[x][y].copy()
        color_index = quantize(old_value, palette)
        output[x][y] = color_index
        if np.isscalar(color_index): new_value = palette[color_index]
        else: new_value = [palette[i] for i in color_index]
        error = old_value - new_value
        if x + 1 < h: IF[x+1][y] += error*7/16
        if (x - 1 >= 0) and (y + 1 < w): IF[x-1][y+1] += error*3/16
        if y + 1 < w: IF[x][y+1] += error*5/16
        if (y + 1 < w) and (x + 1 < h): IF[x+1][y+1] += error*1/16  
    return output

In [ ]:
v = np.array([5, 6])
print(v.shape)
print(type(len(v.shape)))

(2,)
<class 'int'>


**Image Resizing**

You were given two image folders: ```images```, and ```images_200```. The first folder contains the original images at higher resolution, and the second the same images but with the largest size not bigger than 200 pixels.

You need to implement the function ```resizeImage```. This will allow you to run the algorithms on images with the size you prefer. You can take advantage of the OpenCV function ```cv2.resize```.

In [ ]:
def resizeImage(I, maxDim):
    """Given an image, make sure it's no bigger than maxDim on either side"""
    new_image = I
    h = new_image.shape[0]
    w = new_image.shape[1]
    while h > maxDim or w > maxDim:
        new_image = cv2.resize(new_image, (0, 0), fx=0.50, fy=0.50)
        h = new_image.shape[0]
        w = new_image.shape[1]
    return new_image

In [ ]:
h = 20
w = 15
while h > 18 or w > 18:
  h -= 1
  w -= 1
  print(h, w)
print('done')

19 14
18 13
done


**Color Images**

So far your quantozation algorithms have handled grayscale images, now you need to augment them to handle color.

You will need to augment ```quantize(v,palette)``` to handle both a scalar ```v``` and a vector ```v```. In general we can see ```v``` as an n-dimensional vector which will return an n-dimensional vector of palette indices. You can use a for loop, but remember that: (a) broadcasting can take a M-dimensional vector and N-dimensional vector and produce a MxN dimensional matrix; and (b) many functions have an axis
argument.

You will also need to make sure that your version of ```quantizeFloyd(IF,palette)``` can handle images with multiple
channels. Remember that an operation such as ```v = IF[i,j]``` does not copy the value of the color channels, what you get is a reference. So, if you change the data at that reference, this might lead to bugs. You need to make sure you copy the data if necessary.

**Question 5.** For 1 and then for 2 bits per pixel, enbed (show) in the cell below three results of inputs and outputs. Also, make sure you save the same images because you will need to submit them.

**Question 6.** Pick a resolution of your choice, the number of bits of your choiche, and a couple of images of your choice. Generate their dithered versions and embed the inputs and outputs here, and submit them.


**Answer to Question 5:**

Color Quantize Naive 1-Bit Images

![](https://drive.google.com/uc?id=15HOuemkstbEvrgmBq_iF-pzjhHGtBo18) 
![](https://drive.google.com/uc?id=15GbE6dfmQ9yRR3iUDslJmFmkO62Xjoin)
![](https://drive.google.com/uc?id=15LU5dKhxxZdPNef31kZDQeI7IHW_DDCV) 
![](https://drive.google.com/uc?id=15J9c2YNE_iYN1MQ6VSzkVF-sezhGN2PW)
![](https://drive.google.com/uc?id=15VgHDULm5mcaVCtsCkiLxRIDwU_sNh1l) 
![](https://drive.google.com/uc?id=15SAS2vn53OjGmVRdRXxO-x8UjXQuA8VL)

[View all results for quantizeNaive 1-bit](https://drive.google.com/uc?id=16-iR7v534ax0W-fo_7hd1umdlVAiDerE)


Color Quantize Naive 2-Bit Images

![](https://drive.google.com/uc?id=15HOuemkstbEvrgmBq_iF-pzjhHGtBo18) 
![](https://drive.google.com/uc?id=162lshzXN5cnujKJDNl59VOXZ7kYHWLUI)
![](https://drive.google.com/uc?id=15LU5dKhxxZdPNef31kZDQeI7IHW_DDCV) 
![](https://drive.google.com/uc?id=16GfzZ96k0YSpWwoLLcCvrVRa4ZO88nVE)
![](https://drive.google.com/uc?id=15VgHDULm5mcaVCtsCkiLxRIDwU_sNh1l) 
![](https://drive.google.com/uc?id=16m9GkK32kj2eA9YvHXQKQZCzdvzjZSiS)

[View all results for quantizeNaive 2-bit](https://drive.google.com/uc?id=18l0GTvS_9cDIhjEfa8QC2gIBunCpy5mt)


Color Quantize Floyd 1-Bit Images

![](https://drive.google.com/uc?id=15HOuemkstbEvrgmBq_iF-pzjhHGtBo18) 
![](https://drive.google.com/uc?id=1AptQpuV-BEenCpUl-H_Hg0bp4JgkoaTd)
![](https://drive.google.com/uc?id=15LU5dKhxxZdPNef31kZDQeI7IHW_DDCV) 
![](https://drive.google.com/uc?id=1Ah7d0JiGpbkRJBSRvs1xfK_r0X7y1LCw)
![](https://drive.google.com/uc?id=15VgHDULm5mcaVCtsCkiLxRIDwU_sNh1l) 
![](https://drive.google.com/uc?id=1ATb2NiZ2Ho4G8Niic-XdUmgOFbyQVtn6)

[View all results for quantizeNaive 1-bit](https://drive.google.com/uc?id=1Aou3G1aqs1RoH51oTFl20mahppHD5uBI)


Color Quantize Floyd 2-Bit Images

![](https://drive.google.com/uc?id=15HOuemkstbEvrgmBq_iF-pzjhHGtBo18) 
![](https://drive.google.com/uc?id=1C05Fsvq3m8eAhPzWnASWZhMp2azS0URk)
![](https://drive.google.com/uc?id=15LU5dKhxxZdPNef31kZDQeI7IHW_DDCV) 
![](https://drive.google.com/uc?id=1BnYvjbVereF13whU0lK8HlUZ7Bi5Ab7q)
![](https://drive.google.com/uc?id=15VgHDULm5mcaVCtsCkiLxRIDwU_sNh1l) 
![](https://drive.google.com/uc?id=1BRjapR-cfG1eg-AXHjl56wKxM9rOG_rG)

[View all results for quantizeNaive 2-bit](https://drive.google.com/uc?id=1BwqHoSV8gCTgIQbbLpOG7H9H5lXqjGyG)

**Answer to Question 6:**

Color Quantize Floyd 2-Bit My Images

![](https://drive.google.com/uc?id=1Dt23LhJc-N8PDPFAkxT_el2Kl7F6_R99) 
![](https://drive.google.com/uc?id=1Dsbb2DMwFCblA9oI3pghx8XxYXO5nJ6z)
![](https://drive.google.com/uc?id=1DrC710FQhEW0MI3DCUntkJ_9wzUZd-ln) 
![](https://drive.google.com/uc?id=1DqqNR0rrWcAzuGIGWuS9MBV8L890Gau9)
![](https://drive.google.com/uc?id=1E41WZXG7mc5xkPXHd-YlErwFOYWf7rKa) 
![](https://drive.google.com/uc?id=1DzAr_MexIt6weOYoqiG8Ipapx7G23sNc)
![](https://drive.google.com/uc?id=1DxSTLmZ3iVfW4ZAblUI8Z9wD1dXCwdIv) 
![](https://drive.google.com/uc?id=1Duei7wBQxjgnpj8dMZrK_pyqMsDo9m1y)
![](https://drive.google.com/uc?id=1E9KZIvp0uJNg4q2X2MhYVsM0LG327hv3) 
![](https://drive.google.com/uc?id=1E8Z6MCOpnyp1d86iUerhXdZWUszaK5X9)

https://drive.google.com/file/d/1E8Z6MCOpnyp1d86iUerhXdZWUszaK5X9/view?usp=sharing

In [ ]:
# Mount your MIX Google Drive
from google.colab import drive
drive_mount = "/content/drive"
drive.mount(drive_mount)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set the source folder where you have the images
##########################################
# Initially you want to work with the smaller images in images_200
images_path = "/Othercomputers/myPCWindows/schoolStuff [D]/CS678/assignment1/assignment1/images_200"
#images_path = "/Othercomputers/myPCWindows/schoolStuff [D]/CS678/assignment1/assignment1/myImages" # path for my images
source = drive_mount+images_path
print(source)
# Load the images and make sure they are ordered
images = [fn for fn in os.listdir(source) if fn.endswith(".jpg") or fn.endswith(".JPG")]
images.sort()
print(images)

/content/drive/Othercomputers/myPCWindows/schoolStuff [D]/CS678/assignment1/assignment1/images_200
['FHHZ0912.JPG', 'GCLJ7495.JPG', 'IMG_1429.jpg', 'IMG_1740.jpg', 'IMG_1911.jpg', 'IMG_2551.jpg', 'IMG_4644.jpg', 'TCBB4971.JPG', 'VKSG0659.JPG', 'WGQY4638.JPG']


In [ ]:
# Set the target folder where results will be saved
###################################################
target_path = "/Othercomputers/myPCWindows/schoolStuff [D]/CS678/assignment1/assignment1/quantized_images"
target = drive_mount+target_path
if not os.path.exists(target):
  os.mkdir(target)
print(target)

/content/drive/Othercomputers/myPCWindows/schoolStuff [D]/CS678/assignment1/assignment1/quantized_images


In [ ]:
# Pick the algorithm
####################
# You can pick among [quantizeNaive", "quantizeFloyd" ]
algo_name = "quantizeNaive"
if algo_name not in globals():
  print("I don't recognize that algorithm")
  sys.exit(1)
else:
  quantize_algo = globals()[algo_name]
  print(quantize_algo)

# Pick the number of bits used for representing color
#####################################################
# This will define the palette
# The questions require to use 1 and 2 bits per pixel, but you can 
# experiment with other values
palette = coverPalette(2)
print(palette)

# Pick if the image should be converted to grayscale (True/False)
#################################################################
# Initially you want to convert the images to grayscale
grayscale = False


# Set a target output size different than the input, otherwise set it to 0
##########################################################################
# This can upscale the image. This function could be used to generate images
# that compensate for the number of pixels a display assigns to the
# visualization of one image pixel. 
# If you see the images with a browser with a zoom set to 100% you do not
# need this functionality. Just generate the view.html file below for
# the visualization of results
target_size = 0

# You can resize the input to a new resolution, otherwise set it to 0
#####################################################################
# To use this funcitonality you need to have implemented resizeImage
source_size = 300

<function quantizeNaive at 0x7f4fc60688c0>
[1.         0.66666667 0.33333333 0.        ]


In [ ]:
# This is the main loop that quantizes the images

# Keep track of heights; we'll need it since we show some small images and
# will otherwise be at the mercy of the webbrowser resize
heights = {}

# for i,value in enumerate(array):
#    --is shorthand for--
# for i in range(len(array)):
#    value = array[i]
for imageI, image in enumerate(images):
  print("%d/%d" % (imageI, len(images)))

  # Load the image; using opencv we'll usually have to provide uint8
  I = cv2.imread(os.path.join(source, image))
  if source_size > 0:
    I = resizeImage(I, source_size)

  # Convert to [0, 1]
  I = I.astype(np.float) / 255
  if grayscale:
    I = np.mean(I, axis=2)
    print(I.shape)

  # Call the algorithm and reconstruct the image using the palette
  IQ = quantize_algo(I, palette)
  R = reconstructImage(IQ, palette)
  
  # Store the height before we upsample
  # Sometimes it's hard to see the pixels and high-dpi screens screw up
  # Back in my day we were happy if our monitors' width had three digits
  heights[image] = I.shape[0]
  if target_size > 0:
    I, R = upscaleNN(I, target_size), upscaleNN(R, target_size)

  # As a sanity check, we'll write back whatever image we get (including
  # whether we grayscaled it)
  # Beware that you shouldn't tamper with images that you get passed!
  # Note the *255!
  I_path = os.path.join(target, image + "_orig.png")
  cv2.imwrite(I_path, (I * 255).astype(np.uint8))
  R_path = f"{image}_{algo_name}.png"
  R_path = os.path.join(target, R_path)
  cv2.imwrite(R_path, (R * 255).astype(np.uint8))


0/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10


In [ ]:
# Run this cell if you are interested in visualizing all the results in one
# one shot within a webpage. It generates a file view.html that loads all the
# images in a table
view_algos = ["orig", algo_name]

with open(os.path.join(target, "view.html"), "w") as fh:
  fh.write("<html><body><table>")
  fh.write("<tr>")
  for algo in view_algos:
    fh.write(f"<td>{algo}</td>")
  fh.write("</tr>")
  for image in images:
    height = heights[image]
    fh.write("<tr>")
    for algo in view_algos:
      img_path = f"{image}_{algo}.png"
      fh.write(f"<td><img height='{height}' src='{img_path}'></td>")
    fh.write("</tr>")
  fh.write("</table></body></html>")
  fh.close()